In [1]:
from os import name
import pandas as pd
import mygene
import numpy as np
from pybiomart import Server

### Query Ensembl for a list of all Ensembl IDs in the database of human genes. 

In [2]:
server = Server(host='http://www.ensembl.org')

dataset = (server.marts['ENSEMBL_MART_ENSEMBL']
                 .datasets['hsapiens_gene_ensembl'])

ensemblIds = dataset.query(attributes=['ensembl_gene_id'])
ensemblIds = ensemblIds.rename(columns={"Gene stable ID": "ensembl_gene_id"})

ensemblIds.to_csv("../output/agora_ensg_list.txt", index = False, header = False)

ensemblIds.shape

(68324, 1)

### Get info on each gene from mygene

In [3]:
mg = mygene.MyGeneInfo()

bioconductor_gene_info = mg.getgenes(ensemblIds['ensembl_gene_id'], fields=["symbol", "name", "summary", "type_of_gene", "alias"], as_dataframe=True)
bioconductor_gene_info.index.rename("ensembl_gene_id", inplace=True)
bioconductor_gene_info.head()

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-20000...done.
querying 20001-21000...done.
querying 21001-22000...done.
querying 22001-23000...done.
querying 23001-24000...done.
querying 24001-25000...done.
querying 25001-26000...done.
querying 26001-27000...done.
querying 27001-28000...done.
querying 28001-29000...done.
querying 29001-30000...done.
querying 30001-31000...done.
querying 31001-32000...done.
querying 32001-33000...done.
querying 33001-34000...done.
querying 34001-35000...done.
queryin

,_id,_version,alias,name,summary,symbol,type_of_gene,notfound
ensembl_gene_id,,,,,,,,
ENSG00000000003,7105,1.0,"[T245, TM4SF6, TSPAN-6]",tetraspanin 6,The protein encoded by this gene is a member o...,TSPAN6,protein-coding,NaN
ENSG00000000005,64102,1.0,"[BRICD4, CHM1L, TEM]",tenomodulin,This gene encodes a protein that is related to...,TNMD,protein-coding,NaN
ENSG00000000419,8813,1.0,"[CDGIE, MPDS]",dolichyl-phosphate mannosyltransferase subunit...,Dolichol-phosphate mannose (Dol-P-Man) serves ...,DPM1,protein-coding,NaN
ENSG00000000457,57147,1.0,"[PACE-1, PACE1]",SCY1 like pseudokinase 3,This gene encodes a protein with a kinase doma...,SCYL3,protein-coding,NaN
ENSG00000000460,55732,1.0,NaN,chromosome 1 open reading frame 112,NaN,C1orf112,protein-coding,NaN


In [4]:
bioconductor_gene_info[bioconductor_gene_info['notfound'] == True].count()

_id             0
_version        0
alias           0
name            0
summary         0
symbol          0
type_of_gene    0
notfound        9
dtype: int64

In [5]:
bioconductor_gene_info[bioconductor_gene_info['notfound'].isna()].count()

_id             68317
_version        68317
alias           25990
name            49231
summary         24434
symbol          49231
type_of_gene    39583
notfound            0
dtype: int64

In [6]:
bioconductor_gene_info.columns

Index(['_id', '_version', 'alias', 'name', 'summary', 'symbol', 'type_of_gene',
       'notfound'],
      dtype='object')

### We join, and then stardardize our datasets:

In [7]:
gene_table_merged = pd.merge(left=ensemblIds, right=bioconductor_gene_info, how='left', on="ensembl_gene_id")
gene_table_merged.columns = gene_table_merged.columns.str.replace("[#,@,&,*,^,?,(,),%,$,#,!,/]", "", regex = True)
gene_table_merged.columns = gene_table_merged.columns.str.replace("[' ', '-', '.']", "_", regex = True)
gene_table_merged.columns = map(str.lower, gene_table_merged.columns)

gene_table_merged.shape

(68326, 9)

In [8]:
gene_table_merged.columns

Index(['ensembl_gene_id', '_id', '_version', 'alias', 'name', 'summary',
       'symbol', 'type_of_gene', 'notfound'],
      dtype='object')

In [9]:
gene_table_merged.head()

,ensembl_gene_id,_id,_version,alias,name,summary,symbol,type_of_gene,notfound
0,ENSG00000000003,7105,1.0,"[T245, TM4SF6, TSPAN-6]",tetraspanin 6,The protein encoded by this gene is a member o...,TSPAN6,protein-coding,NaN
1,ENSG00000000005,64102,1.0,"[BRICD4, CHM1L, TEM]",tenomodulin,This gene encodes a protein that is related to...,TNMD,protein-coding,NaN
2,ENSG00000000419,8813,1.0,"[CDGIE, MPDS]",dolichyl-phosphate mannosyltransferase subunit...,Dolichol-phosphate mannose (Dol-P-Man) serves ...,DPM1,protein-coding,NaN
3,ENSG00000000457,57147,1.0,"[PACE-1, PACE1]",SCY1 like pseudokinase 3,This gene encodes a protein with a kinase doma...,SCYL3,protein-coding,NaN
4,ENSG00000000460,55732,1.0,NaN,chromosome 1 open reading frame 112,NaN,C1orf112,protein-coding,NaN


### Clean the data
Fix NULL values in the "alias" field and make sure every alias value is a list, not a string.

In [10]:
# NULL alias values become empty lists
for row in gene_table_merged.loc[gene_table_merged['alias'].isnull(), 'alias'].index:
    gene_table_merged.at[row, 'alias'] = []

# Some alias values are a single string, not a list. Turn them into lists here.
gene_table_merged['alias'] = gene_table_merged['alias'].apply(lambda cell: cell if isinstance(cell, list) else [cell])

### Remove duplicate Ensembl IDs from the list. 

Duplicates in the list typically have the same Ensembl ID but different gene symbols. There's not a good way to reconcile this, so just use the first entry in the list for each ensembl ID and discard the rest, which is what the Agora front end does. The gene symbols of duplicate rows are then added as aliases to the matching unique row.

In [11]:
# duplicated() will return true if the ID is a duplicate and is not the first one to appear the list. 
dupes = gene_table_merged['ensembl_gene_id'].duplicated()
dupe_vals = gene_table_merged[dupes]

# Remove duplicates from the list
gene_table_merged = gene_table_merged[dupes == False].reset_index()

# For each duplicate row, add its symbol as an alias
for row in dupe_vals.index:
    match = gene_table_merged['ensembl_gene_id'] == dupe_vals['ensembl_gene_id'][row]
    match_ind = gene_table_merged[match].index[0] # There should only be one row

    # Add the duplicate's symbol to the alias list
    gene_table_merged.at[match_ind, 'alias'].append(dupe_vals['symbol'][row])
    
    # Make sure we didn't add duplicate aliases
    gene_table_merged.at[match_ind, 'alias'] = list(set(gene_table_merged.at[match_ind, 'alias']))

print(gene_table_merged.shape)

(68324, 10)


### Write to a file
This will get uploaded to Synapse as [syn25953363](https://www.synapse.org/#!Synapse:syn25953363).

In [12]:
gene_table_merged.to_feather('../output/gene_table_merged_GRCh38.p13.feather')